# Example Usage of the places_client

### 1. Install and import the package

In [ ]:
#%pip install places_client
from places_client.places_client import PlacesClient

### 2. Create a client for the API

In [ ]:
# store your api token as "CDC_API_TOKEN" in the .env file
load_dotenv()
token = os.getenv('CDC_API_TOKEN')
client = PlacesClient(token)

### 3. Retrieve a certain release of county-level PLACES data 

In [ ]:
# get the 2024 release of county-level PLACES dataset
df_2024 = client.get_county_data(release="2024")
df_2024.head()